In [ ]:
"""This is basically a dataflow programming language."""

In [22]:
def get_keys_from_values(dic: dict, values: list):
    key_lst = list(dic.keys())
    val_lst = list(dic.values())
    return [key_lst[val_lst.index(val)] for val in values]

In [321]:
class FunctionObj:
    def __init__(self, func: callable, input_types: list[type], output_types: list[type], func_name='function'):
        self.name = func_name
        self.func = func
        self.input_types = input_types
        self.output_types = output_types
        self.validate()
    
    def validate(self):
        #TODO: implement.
        pass

    def run(self, *args):
        return self.func(*args)
    
class ObjNode:
    def __init__(self, obj_type: type):
        #TODO: should i add an id number in the object itself?
        self.name = f'{obj_type} obj'
        self.obj_type: type = obj_type
        self.func_children: list[tuple[FuncNode,int]] = []
        self.func_parent: FuncNode = None
        self.data = None
        self.comp_graph_level = 0
        self.iterator_authority = None

    def update_data(self, data):
        self.data = data

    def append_data(self, data):
        if not self.obj_type.__origin__ == list:
            raise ValueError('object not iterable!')
        if self.data == None:
            self.data = [data]
        else:
            self.data.append(data)
    
    def contains_data(self):
        return self.data is not None

class FuncNode:
    def __init__(self, func_obj: FunctionObj):
        #TODO: should i add an id number in the object itself?
        self.name = func_obj.name
        self.func_obj: FunctionObj = func_obj
        self.input_obj_nodes: list[ObjNode] = [None for i in range(len(self.func_obj.input_types))]
        self.output_obj_nodes: list[ObjNode] = [ObjNode(obj_type) for obj_type in self.func_obj.output_types]
        for obj in self.output_obj_nodes:
            obj.func_parent = self
        self.comp_graph_level = 0
        self.iterator_authority = None
    
    def connect_input(self, input_index: int, obj_node: ObjNode):
        if self.input_obj_nodes[input_index] == obj_node:
            raise Exception('node already connected to the same input index!')
        self.input_obj_nodes[input_index] = obj_node
        obj_node.func_children.append((self, input_index))
    
    def disconnect_input(self, input_index):
        if self.input_obj_nodes[input_index] == None:
            return
        input_obj = self.input_obj_nodes[input_index]
        input_obj.func_children.remove((self, input_index))
        self.input_obj_nodes[input_index] = None
    
    def run(self):
        input_data = [obj.data for obj in self.input_obj_nodes]
        out_data = self.func_obj.run(*input_data)
        for i, obj in enumerate(self.output_obj_nodes):
            obj.update_data(out_data[i])
        
    def update_level(self):
        input_nodes = [node for node in self.input_obj_nodes if node is not None]
        max_level = 0
        for node in input_nodes:
            if node.iterator_authority == None or node.iterator_authority == self.iterator_authority:
                max_level = max(max_level, node.comp_graph_level)
            elif node.iterator_authority != self.iterator_authority:
                max_level = max(max_level, node.iterator_authority.comp_graph_level)
        self.comp_graph_level = max_level + 1
        for output_node in self.output_obj_nodes:
            output_node.comp_graph_level = self.comp_graph_level + 1
        
        if self.iterator_authority:
            self.iterator_authority.update_level()


appending_function = FunctionObj(lambda x: [x], [object], [object], 'appending_function')
class AppendFuncNode(FuncNode):
    def __init__(self, appending_function):
        self.name = appending_function.name
        self.func_obj: FunctionObj = appending_function
        self.input_obj_nodes: list[ObjNode] = [None]
        self.output_obj_nodes: list[ObjNode] = [ObjNode(list[object])]
        for obj in self.output_obj_nodes:
            obj.func_parent = self
        self.comp_graph_level = 0
        self.iterator_authority = None
    
    def run(self):
        input_data = [obj.data for obj in self.input_obj_nodes]
        out_data = self.func_obj.run(*input_data)
        for i, obj in enumerate(self.output_obj_nodes):
            obj.append_data(out_data[i])
        


class IteratorNode:
    # implementing first without accumulator objects.
    # Nicely done. this is basically complete.
    # things to improve and add:
    def __init__(self, obj_input_type):
        self.input_type = obj_input_type
        self.comp_graph_level = 0
        self.iterated_obj: ObjNode = None
        self.iteration_root_obj: ObjNode = None
        self.obj_nodes: list[ObjNode] = []
        self.func_nodes: list[ObjNode] = []
        self.iterator_nodes = []
        self.iterator_authority: IteratorNode = None

    def connect_iterated_obj(self, obj_node: ObjNode):
        if self.iterated_obj == obj_node:
            raise Exception('node already connected to the same input index!')
        self.iterated_obj = obj_node
        obj_node.func_children.append((self, 0))

        self.iteration_root_obj = ObjNode(obj_node.obj_type.__args__[0])
        self.obj_nodes.append(self.iteration_root_obj)
        self.iteration_root_obj.iterator_authority = self
    
    def add_func_node(self, func_node: FuncNode):
        func_node.iterator_authority = self
        self.func_nodes.append(func_node)
        for obj in func_node.output_obj_nodes:
            obj.iterator_authority = self
            self.obj_nodes.append(obj)
    
    def add_iterator_node(self, iterator_node: 'IteratorNode'):
        iterator_node.iterator_authority = self
        self.iterator_nodes.append(iterator_node)

    def run(self):
        if (self.iterated_obj is None) or (not isinstance(self.iterated_obj.data, list)):
            raise Exception("IteratorNode requires an input ObjNode containing a list.")
        
        # Clear data in all nodes managed by the iterator before starting the iteration
        for obj in self.obj_nodes:
            obj.update_data(None)
        
        # Iterate through the elements in the list stored in the iterated object node
        for element in self.iterated_obj.data:
            # Update the iteration root object with the current element
            self.iteration_root_obj.update_data(element)
            # Run all function nodes in the iterator's context, ordered by their comp_graph_level
            for func_node in sorted(self.func_nodes + self.iterator_nodes, key=lambda fn: fn.comp_graph_level):
                func_node.run()
        return
    

    def update_level(self):
        self.iteration_root_obj.comp_graph_level = self.iterated_obj.comp_graph_level + 2
        self.comp_graph_level = max([obj.comp_graph_level for obj in self.obj_nodes])
        if self.iterator_authority != None:
            self.iterator_authority.update_level()


class ComputationalNetwork:
    # TODO: write __str__, __repr__, etc. for all classes
    
    def __init__(self):
        self.input_obj_nodes = dict()  # Stores input nodes with their unique IDs or references
        self.obj_nodes = dict()        # Stores all ObjNodes with their unique IDs or references
        self.func_nodes = dict()       # Stores all FuncNodes with their unique IDs or references
        self.iterator_nodes = dict()
        self.conditional_nodes = dict()
        self.output_object: ObjNode = None
    
    def _create_unique_id(self):
        return max(list(self.func_nodes.keys()) + list(self.obj_nodes.keys()) + list(self.iterator_nodes.keys()), default=0) + 1
    
    def appoint_solution_object(self, obj_node: ObjNode):
        #TODO: this should be in a new inherited class
        self.output_object = obj_node
    
    def add_func_node(self, func_obj: FunctionObj, input_node_ids: list[int], iterator_authority: IteratorNode = None):
        if func_obj.name == 'appending_function':
            func_node = AppendFuncNode(func_obj)
        else:
            func_node = FuncNode(func_obj)

        if iterator_authority != None:
            iterator_authority.add_func_node(func_node)

        # Connect input nodes
        for i, obj_id in enumerate(input_node_ids):
            func_node.connect_input(i, self.obj_nodes[obj_id])

        # Generate a unique ID for the function node
        func_node_id = self._create_unique_id()
        self.func_nodes[func_node_id] = func_node
        
        # Add output nodes to the obj_nodes dictionary
        for i, output_node in enumerate(func_node.output_obj_nodes):
            obj_node_id = func_node_id + i + 1
            self.obj_nodes[obj_node_id] = output_node
        
        func_node.update_level()
        return func_node
    
    def add_input_obj_node(self, obj_type: type):
        node = ObjNode(obj_type)
        node_id = self._create_unique_id()
        self.input_obj_nodes[node_id] = node
        self.obj_nodes[node_id] = node  # Also include it in the general obj_nodes dictionary
        return node
    
    def add_iterator_node(self, input_node_id, iterator_authority: IteratorNode = None):
        iterator_node = IteratorNode(self.obj_nodes[input_node_id].obj_type)

        if iterator_authority != None:
            iterator_authority.add_iterator_node(iterator_node)
        iterator_node.connect_iterated_obj(self.obj_nodes[input_node_id])

        iterator_node_id = self._create_unique_id()
        self.iterator_nodes[iterator_node_id] = iterator_node
        iteration_root_obj_id = self._create_unique_id()
        self.obj_nodes[iteration_root_obj_id] = iterator_node.iteration_root_obj
        iterator_node.update_level()

        return iterator_node

    def clean_data(self):
        for obj in self.obj_nodes.values():
            obj.data = None
    
    def run(self, inputs: dict):
        # Assign initial values to input nodes
        for node_id, value in inputs.items():
            if node_id not in self.input_obj_nodes:
                raise ValueError("Provided node ID is not a valid input node.")
            self.input_obj_nodes[node_id].update_data(value)
        
        # TODO: accomodate iterator node.
        
        free_nodes = [node for node in list(self.func_nodes.values()) + list(self.iterator_nodes.values()) if node.iterator_authority == None]
        for func_node in sorted(free_nodes, key=lambda func_node: func_node.comp_graph_level):
            func_node.run()
    
    def run_specified_nodes(self, nodes):
        # don't implement right now
        pass

    def get_all_nodes(self):
        nodes_dict = self.func_nodes.copy()
        nodes_dict.update(self.obj_nodes)
        nodes_dict.update(self.iterator_nodes)
        return nodes_dict
    
    def delete_func_node(self, id_number):
        if id_number not in self.func_nodes:
            #TODO: rewrite to make it an exception.
            return
            #raise IndexError("function node doesn't exist.")
        func_node: FuncNode = self.func_nodes[id_number]
        for index in range(len(func_node.input_obj_nodes)):
            func_node.disconnect_input(index)
        
        obj_children_ids = get_keys_from_values(self.obj_nodes, func_node.output_obj_nodes)
        for obj_child_id in obj_children_ids:
            obj: ObjNode = self.obj_nodes[obj_child_id]
            func_children_ids = get_keys_from_values(self.func_nodes, [p[0] for p in obj.func_children])
            for child_func_node_id in func_children_ids:
                self.delete_func_node(child_func_node_id)

            del self.obj_nodes[obj_child_id]
        del self.func_nodes[id_number]
    
    def get_data_dic(self):
        return {node_id: node.data for node_id, node in self.obj_nodes.items() if node.contains_data()}

    def get_dependent_func_ids(self, func_id_number):
        # don't implement right now
        pass

    def get_basic_structure_dic(self):
        dic = {}
        for obj_id, obj in self.obj_nodes.items():
            dic[obj_id] = list(get_keys_from_values(self.func_nodes, [f for f, id in obj.func_children]))

        for func_id, func in self.func_nodes.items():
            dic[func_id] = list(get_keys_from_values(self.obj_nodes, func.output_obj_nodes))
        
        return dic
    
    def get_full_structure_dic(self):
        dic = {}
        for obj_id, obj in self.obj_nodes.items():
            dic[(obj_id, obj)] = [(f_id, self.func_nodes[f_id]) for f_id in get_keys_from_values(self.func_nodes, [f for f, id in obj.func_children])]

        for func_id, func in self.func_nodes.items():
            dic[(func_id, func)] = [(obj_id, self.obj_nodes[obj_id]) for obj_id in get_keys_from_values(self.obj_nodes, func.output_obj_nodes)]
        pass



In [322]:
def example1():
    network = ComputationalNetwork()
    func1 = FunctionObj(lambda x: [x**2], input_types=[int], output_types=[int],func_name='^2')
    func2 = FunctionObj(lambda x,y: [x+y], input_types=[int, int], output_types=[int], func_name='+')
    func3 = FunctionObj(lambda x,y: [x+y, x-y], input_types=[int, int], output_types=[int,int], func_name='f3')
    
    network.add_input_obj_node(int)
    network.input_obj_nodes
    network.add_func_node(func1,[1])
    network.add_func_node(func2, [1,3])
    network.add_func_node(func3, [3,5])
    network.add_func_node(func1, [5])
    print(network.get_basic_structure_dic())
    print(network.run({1:2}))
    print(network.get_data_dic())
    #print('\n'.join([str((x, x.comp_graph_level)) for x in list(network.get_all_nodes().values())]))
    network.delete_func_node(9)
    print(network.get_basic_structure_dic())
example1()

{1: [2, 4], 3: [4, 6], 5: [6, 9], 7: [], 8: [], 10: [], 2: [3], 4: [5], 6: [7, 8], 9: [10]}
None
{1: 2, 3: 4, 5: 6, 7: 10, 8: -2, 10: 36}
{1: [2, 4], 3: [4, 6], 5: [6], 7: [], 8: [], 2: [3], 4: [5], 6: [7, 8]}


In [323]:
def iterator_node_example():
    iterator_node = IteratorNode(list[int])
    iterated_obj = ObjNode(list[int])
    iterated_obj.update_data([1,4,5])
    iterator_node.connect_iterated_obj(iterated_obj)

    def f(x):
        return [x**2]

    f_obj = FunctionObj(f, [int], [int])
    f_node = FuncNode(f_obj)
    iterator_node.iteration_root_obj.obj_type
    f_node.connect_input(0, iterator_node.iteration_root_obj)
    iterator_node.add_func_node(f_node)
    f_node.output_obj_nodes[0].iterator_authority
    iterated_obj.update_data([1,4,5])
    iterator_node.run()
    print([obj.data for obj in iterator_node.obj_nodes])
iterator_node_example()

[5, 25]


In [332]:
network = ComputationalNetwork()
func1 = FunctionObj(lambda x: [x**2], input_types=[int], output_types=[int],func_name='^2')
func2 = FunctionObj(lambda x,y: [x+y], input_types=[int, int], output_types=[int], func_name='+')
func3 = FunctionObj(lambda x,y: [x+y, x-y], input_types=[int, int], output_types=[int,int], func_name='f3')
appending_function = appending_function

network.add_input_obj_node(list[int])
iter1 = network.add_iterator_node(1)
network.get_all_nodes()
network.add_func_node(func1, [3], iterator_authority=network.iterator_nodes[2])
network.add_func_node(appending_function,[5],iterator_authority=network.iterator_nodes[2])
iter2 = network.add_iterator_node(7, network.iterator_nodes[2])
network.add_func_node(func2,[3,9], iterator_authority=iter2)
network.add_func_node(appending_function, [11], iterator_authority=iter1)
network.get_all_nodes()


{4: <__main__.FuncNode at 0x110988af0>,
 6: <__main__.AppendFuncNode at 0x110a156f0>,
 10: <__main__.FuncNode at 0x110989930>,
 12: <__main__.AppendFuncNode at 0x11098b5b0>,
 1: <__main__.ObjNode at 0x126f301c0>,
 3: <__main__.ObjNode at 0x126f30f10>,
 5: <__main__.ObjNode at 0x11098a920>,
 7: <__main__.ObjNode at 0x11098bd90>,
 9: <__main__.ObjNode at 0x11098b8e0>,
 11: <__main__.ObjNode at 0x11098b310>,
 13: <__main__.ObjNode at 0x11098a560>,
 2: <__main__.IteratorNode at 0x126f30c10>,
 8: <__main__.IteratorNode at 0x110a15750>}

In [333]:
network.run({1:[i for i in range(11)]})
network.get_data_dic()

{1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 3: 10,
 5: 100,
 7: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100],
 9: 100,
 11: 110,
 13: [0, 2, 6, 12, 20, 30, 42, 56, 72, 90, 110]}

In [334]:
def visualize_network(network: ComputationalNetwork):
    pass

In [246]:
def get_max(lst):
    return [max(lst)]

get_max_func = FunctionObj(get_max,[list[int]], [int])

def init_list():
    return [list()]

initialize_list = FunctionObj(init_list, [], [list])

def append_to_list(lst: list, integer: int):
    lst = lst.copy()
    lst.append(integer)
    return [lst]

def iterate(subnetwork: ComputationalNetwork, condition):
    #run
    pass

class IntegerSequenceTask:
    def __init__(self, train: list[tuple[list[int],list[int]]], test: tuple[list[int], list[int]]):
        self.train = train #for example: [([1,4,2,9], [1,2,4,9]), ...]
        self.test = test #for example: ([1,4,2,9], [1,2,4,9])
    

class ProtoKubernetes:
    """a prototype of a more sophisticated class that will be called Kubernetes.
    Objects in this class are responsible for using ComputationalNetwork objects
    to find network solutions to problems.
    """
    def __init__(self):
        self.known_functions = []
        self.known_networks = []
        self.solved_tasks = []
        self.unsolved_tasks = []
        self.concept_library = dict()
        self.current_task = None
    
    def add_known_function(self, func_obj: FunctionObj):
        self.known_functions.append(func_obj)
    
    def search_solution(self, task, max_functions=5):
        # TODO: since my network object doesn't really care about the depth level per se, 
        # (i.e. each level of depth can be used in any higher level of depth), maybe the 
        # limit should be the number of functions and not the depth.
        solution_candidates = []
        for example_input, example_output in task.train:
            net_candidate = ComputationalNetwork()
            net_candidate.add_input_obj_node(list[int])


In [6]:
# This cell contains ideas that will take very hard work to bring into fruition. very exciting ideas but will take time to fully realize.

class Task:
    def __init__(self, train: list[tuple[object,object]], test: object, task_type):
        self.train = train
        self.test = test
        self.task_type = task_type
        self.solved = False
        self.solution_func = None

class ProblemSolvingProtocol:
    pass

class EvaluationControler:
    pass

class Kubernetes:
    protocols = []
    functions = []
    known_objects = []
    concept_library = []

    def __init__(self):
        self.task = None

    def assign_task(self, task):
        self.task = task
        pass

    def create_subtask(self, subtask):
        pass

    def create_sub_kubernetes(self, protocol):
        pass

    def search_solution(self, task_id, protocol):
        pass

In [65]:
list[int].__args__[0]

int